In [1]:
import pandas as pd
from io import StringIO
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
#Get the spam data collection 
df_spamCollection=pd.read_csv('/content/sample_data/SpamCollection',sep='\t',names=['response','message'])

In [5]:
df_spamCollection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df_spamCollection.describe()


,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [7]:
#view response 
df_spamCollection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [8]:
#Verify length of the messages and also add it as a new column 
df_spamCollection['length']=df_spamCollection['message'].apply(len)

In [9]:
df_spamCollection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [10]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
  no_punctuation=[char for char in mess if char not in string.punctuation]
  no_punctuation=''.join(no_punctuation)
  return[word for word in no_punctuation.split() if word.lower not in stopwords.words('english')]

In [11]:
df_spamCollection['message'].head(5).apply(message_text_process)

0    [Go, until, jurong, point, crazy, Available, o...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, in, 2, a, wkly, comp, to, win, F...
3    [U, dun, say, so, early, hor, U, c, already, t...
4    [Nah, I, dont, think, he, goes, to, usf, he, l...
Name: message, dtype: object

In [12]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
#use bag of words by applying the function and fit the data into it
bag_of_words_transformer=CountVectorizer(analyzer=message_text_process).fit(df_spamCollection['message'])

In [14]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bag_of_words_transformer.vocabulary_))

11747


In [15]:
message_bagofwords=bag_of_words_transformer.transform(df_spamCollection['message'])

In [21]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(message_bagofwords)

In [23]:
#print shape of the tfidf 
message_tfidf=tfidf_transformer.transform(message_bagofwords)
message_tfidf.shape

(5572, 11747)

In [24]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(message_tfidf,df_spamCollection['response'])

In [31]:
#check model for the predicted and expected value say for message#2 and message#5
message=df_spamCollection['message'][2]
bag_of_words_for_message=bag_of_words_transformer.transform([message])
tfidf=tfidf_transformer.transform(bag_of_words_for_message)

print('predicted',spam_detect_model.predict(tfidf)[0])
print('expected',df_spamCollection.response[2])

('predicted', 'spam')
('expected', 'spam')


In [32]:
message=df_spamCollection['message'][5]
bag_of_words_for_message=bag_of_words_transformer.transform([message])
tfidf=tfidf_transformer.transform(bag_of_words_for_message)

print('predicted',spam_detect_model.predict(tfidf)[0])
print('expected',df_spamCollection.response[5])

('predicted', 'ham')
('expected', 'spam')
